In [1]:
import numpy as np
import math
from IPython.core.display import display, HTML

In [629]:
class Grid:
    def __init__(self, rows, columns, labelVec = None):
        self.rows = rows
        self.columns = columns
        
        if(labelVec == None):
            self.labelVec = list(range(1,rows*columns+1))
        else:
            self.labelVec = labelVec.copy()
        
    def display(self):
        table = "<table>{0}</table>"
        row = "<tr>{0}</tr>"
        cell = "<td>{0}</td>"

        cells = ""
        rows = "" 
        for i in range(1,self.rows*self.columns+1):
            cells += cell.format(self.labelVec[i-1])
            if((i%self.rows) == 0):
                rows += row.format(cells)
                cells = ""

        display(HTML(table.format(rows)))
        
    def __repr__(self):
        self.display()
        return "{0}x{1} grid".format(self.rows,self.columns)
    
    # Obviously could be simplified if we just used nested loops, not sure why
    # I felt the need to do it in this way. Maybe the math is getting to me. 
    def V(self):
        if(self.rows % 2 != 0 or self.rows % 2 != 0):
            itter = self.rows*math.floor( math.sqrt(self.rows*self.columns) / 2 )
        else:
            itter = self.rows*math.floor(self.columns/2)
        for i in range(itter):
            overLoc = (self.rows-1+self.rows*(i%self.rows))-math.floor(i/self.rows)
            leftLoc = math.floor(i/self.rows)+i%self.rows+((self.rows-1)*(i%self.rows))
            tmp = self.labelVec[leftLoc]
            
            self.labelVec[leftLoc] = self.labelVec[overLoc]
            self.labelVec[overLoc] = tmp
        
        return self
                

    
    # Horizontal Symmetry 
    def H(self):
        if(self.rows % 2 != 0 or self.rows % 2 != 0):
            itter = self.rows+self.rows*math.floor( math.sqrt(self.rows*self.columns) / 2 )
        else:
            itter = self.rows*math.floor(self.columns/2)
            
        for i in range(itter):
            overLoc = (self.rows*self.columns-self.rows*(math.floor(i/self.rows)+1)+(i%(self.rows)))
            tmp = self.labelVec[i]
            self.labelVec[i] = self.labelVec[overLoc]
            self.labelVec[overLoc] = tmp
            
        return self

In [641]:
g = Grid(3,3)
g.H().V()

9,8,7
6,5,4
3,2,1


3x3 grid

In [639]:
g.H().V()

1,2,3
4,5,6
7,8,9


3x3 grid

In [611]:
z = Grid(4,4)
z

1,2,3,4
5,6,7,8
9,10,11,12
13,14,15,16


4x4 grid

In [480]:
g.H()
g

7,8,9
4,5,6
1,2,3


3x3 grid

In [481]:
g.V()
g

IndexError: list index out of range

In [454]:
5/2

2.5

In [79]:
del g 